In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp

In [2]:
# Import data
resale = pd.read_excel('Ticket_Data.xlsx', sheetname='Resale Estimates')
tier_price = pd.read_excel('Ticket_Data.xlsx', sheetname='Tier Prices')
all_star_games = pd.read_excel('Ticket_Data.xlsx', sheetname='All Star Games')

In [3]:
# Merge games with prices
data = pd.merge(resale, tier_price, on=['Tier'])

if (len(resale)==len(data)):
    print ('Merge Good')
else:
    print ('Merge Gone Bad')

Merge Good


In [4]:
# Merge games with all star plan
all_star_games['All_Star'] = 1
all_star_games['Price_All_Star'] = 650.0/13.0
data = pd.merge(data, all_star_games[['Date', 'All_Star', 'Price_All_Star']], on='Date', how='left')
data['All_Star']  = data['All_Star'].fillna(0)
data['Price_All_Star']  = data['Price_All_Star'].fillna(0)

In [5]:
def optimize_games(game_list, n_games, price_var, opt_var, seller_perc = .1, bonus_game = True, all_star = False,
                   gold_games_max = None, white_red_games_min = 0, red_games_min = 0):
    
    # Adjust resale for seller costs
    game_list['Resale'] = game_list['Resale'] * (1-seller_perc)
    
    # Calculate Profit
    game_list['Profit'] = game_list['Resale'] - game_list[price_var]
    
    # Create dummy vars
    game_list = pd.get_dummies(game_list, columns=['Tier'])
    game_list['Tier_W_R'] = game_list['Tier_W'] + game_list['Tier_R']
    game_list['Count'] = 1
    
    # Create solver object
    solver = pywraplp.Solver('SolveIntegerProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # create x array
    x_list = {}
    
    # variable for each game.  1 is buy.  0 is not
    for i in range(len(game_list)):
        x_list[i] = solver.IntVar(0.0, 1.0, 'x'+str(i))
        
    # Adds constraints
    constraint_count = solver.Constraint(n_games, n_games)
    for i in range(len(game_list)):
        constraint_count.SetCoefficient(x_list[i], int(game_list.loc[i, 'Count']))

    # Constraint for max number of gold games
    if gold_games_max is not None:
        constraint_g = solver.Constraint(0, gold_games_max)
        for i in range(len(game_list)):
            constraint_g.SetCoefficient(x_list[i], int(game_list.loc[i, 'Tier_G']))
        
    # Constraint for min number of white or red games
    constraint_wr = solver.Constraint(white_red_games_min, solver.infinity())
    for i in range(len(game_list)):
        constraint_wr.SetCoefficient(x_list[i], int(game_list.loc[i, 'Tier_W_R']))
        
    # Constraint for min number of red games
    constraint_r = solver.Constraint(red_games_min, solver.infinity())
    for i in range(len(game_list)):
        constraint_r.SetCoefficient(x_list[i], int(game_list.loc[i, 'Tier_R']))
    
    # Adds constraints for all star games
    if all_star:
        constraint_all_star = solver.Constraint(13, 13)
        for i in range(len(game_list)):
            constraint_all_star.SetCoefficient(x_list[i], int(game_list.loc[i, 'All_Star']))
    
   # Set objective     
    obj = solver.Objective()
    for i in range(len(game_list)):
        obj.SetCoefficient(x_list[i], int(game_list.loc[i, 'Profit']))
    obj.SetMaximization()
    
    # Solve problem
    result_status = solver.Solve()
    results = game_list.copy()

    for i in range(len(game_list)):
        results.set_value(i, 'Buy', x_list[i].solution_value())
        
    buy = results.loc[results['Buy']==1]
    
    # subtracts bonus game, which price is the min price
    bonus_price = 0
    if all_star:
        bonus_price = 0
    elif white_red_games_min is not None and buy['Tier_W'].sum()>0:
        bonus_price = buy.loc[buy['Tier_W']==1][price_var].mean()
    elif white_red_games_min is not None and buy['Tier_R'].sum()>0:
        bonus_price = buy.loc[buy['Tier_R']==1][price_var].mean()
    elif red_games_min is not None and buy['Tier_R'].sum()>0:
        bonus_price = buy.loc[buy['Tier_R']==1][price_var].mean()

    print ('Profit', buy['Profit'].sum() + bonus_price)
    print ('Price', (buy[price_var].sum() - bonus_price))
    print ('ROI', (buy['Profit'].sum() + bonus_price) / (buy[price_var].sum() - bonus_price))
    
    return buy

In [6]:
games_6 = optimize_games(data, 7, 'Price_6', 'Profit', gold_games_max=1, red_games_min=1)

Profit 193.0
Price 473.0
ROI 0.4080338266384778


In [7]:
games_6

,Date,Opponent,Resale,Price_13,Price_6,Quality,All_Star,Price_All_Star,Profit,Tier_B,Tier_G,Tier_R,Tier_W,Tier_W_R,Count,Buy
9,2019-01-11,Bucks,54.0,48,50,3,0.0,0.0,4.0,0,0,0,1,1,1,1.0
10,2019-01-13,Raptors,54.0,48,50,3,0.0,0.0,4.0,0,0,0,1,1,1,1.0
23,2018-11-26,Rockets,90.0,67,70,2,1.0,50.0,20.0,1,0,0,0,0,1,1.0
24,2018-12-12,Boston,99.0,67,70,2,0.0,0.0,29.0,1,0,0,0,0,1,1.0
27,2019-04-09,Boston,126.0,67,70,2,1.0,50.0,56.0,1,0,0,0,0,1,1.0
29,2018-11-20,Clippers,36.0,43,45,4,0.0,0.0,-9.0,0,0,1,0,1,1,1.0
38,2018-12-16,Lakers,207.0,143,168,1,1.0,50.0,39.0,0,1,0,0,0,1,1.0


In [8]:
games_13 = optimize_games(data, 14, 'Price_13', 'Profit', gold_games_max=2, white_red_games_min=1)

Profit 160.20000000000005
Price 909.0
ROI 0.17623762376237628


In [9]:
games_13

,Date,Opponent,Resale,Price_13,Price_6,Quality,All_Star,Price_All_Star,Profit,Tier_B,Tier_G,Tier_R,Tier_W,Tier_W_R,Count,Buy
4,2018-11-18,Blazers,40.50,48,50,3,0.0,0.0,-7.50,0,0,0,1,1,1,1.0
7,2018-12-28,Bulls,48.60,48,50,3,1.0,50.0,0.60,0,0,0,1,1,1,1.0
8,2018-12-29,Hornets,44.55,48,50,3,0.0,0.0,-3.45,0,0,0,1,1,1,1.0
9,2019-01-11,Bucks,48.60,48,50,3,0.0,0.0,0.60,0,0,0,1,1,1,1.0
10,2019-01-13,Raptors,48.60,48,50,3,0.0,0.0,0.60,0,0,0,1,1,1,1.0
12,2019-02-23,Pacers,40.50,48,50,3,1.0,50.0,-7.50,0,0,0,1,1,1,1.0
17,2019-03-23,Heat,40.50,48,50,3,0.0,0.0,-7.50,0,0,0,1,1,1,1.0
21,2018-11-02,Thunder,64.80,67,70,2,1.0,50.0,-2.20,1,0,0,0,0,1,1.0
22,2018-11-24,Pelicans,60.75,67,70,2,0.0,0.0,-6.25,1,0,0,0,0,1,1.0
23,2018-11-26,Rockets,81.00,67,70,2,1.0,50.0,14.00,1,0,0,0,0,1,1.0


In [10]:
games_all_star = optimize_games(data, 14, 'Price_All_Star', 'Profit', white_red_games_min=1, all_star=True)

Profit 261.25000000000006
Price 650.0
ROI 0.401923076923077
